In [73]:
using Plots
gr()

Plots.GRBackend()

In [74]:
using MLDataPattern
using LIBSVM
using AxisArrays
using JuMP
using ProgressMeter

In [75]:
@showprogress 1 "foo" for i in 1:50
    sleep(0.1)
end

foo100% Time: 0:00:05


In [76]:
include("main.jl")

c

In [77]:
double_integrator = c.MPC.discretize(c.MPC.CTLinearSytstem([0. 1; 0 0], [0. 1]'), 0.1)
time = Axis{:time}(linspace(0, 9 * double_integrator.Δt, 10))
side = Axis{:side}([:left, :right])
model = c.MPC.create_model(double_integrator, time, side);
numresults = 10000
results = c.Record[]
@showprogress 1 "gathering data... " for i in 1:numresults
    push!(results, c.Record(model))
end

gathering data... 100%  ETA: 0:00:01

In [78]:
examples = convert(Vector{c.Example}, results);

labels = [e.best_action for e in examples]
data = hcat([vcat(e.contact, e.duals) for e in examples]...);

In [79]:
(data_train, labels_train), (data_test, labels_test) = splitobs((data, labels), 0.5);

In [80]:
@time svm = svmtrain(data_train, labels_train; cost=100., probability=true)
@time predicted_labels, decision_values = svmpredict(svm, data_test)

 13.726812 seconds (4.58 k allocations: 13.288 MB, 0.04% gc time)
  3.878712 seconds (4.52 k allocations: 22.124 MB, 0.09% gc time)


([2,16,2,13,1,18,3,1,3,2  …  15,11,9,7,2,1,3,14,4,1],
[0.00311711 0.0672858 … 0.00419944 0.7895; 0.0043126 0.000891773 … 0.00516554 0.00180899; … ; 0.000254341 0.00127563 … 0.00295882 0.000619531; 0.000178945 0.000960092 … 0.00175041 0.00044275])

In [81]:
mean(predicted_labels .== labels_test)

0.6514

In [82]:
q0 = 2 * rand() - 1
v0 = 4 * rand() - 2
qlimb0 = 2 * rand() - 1
state = c.MPC.State(q0, v0, qlimb0)
contact = AxisArray(rand(Bool, 2, length(time)), side, time)

2-dimensional AxisArray{Bool,2,...} with axes:
    :side, Symbol[:left,:right]
    :time, linspace(0.0,0.9,10)
And data, a 2×10 Array{Bool,2}:
 false  false  true  false  false  false   true   true   true  false
 false   true  true   true  false   true  false  false  false  false

In [83]:
function svmbeam(svm::LIBSVM.SVM, model::c.MPC.MPCModel, state::c.MPC.State, contact)
    active_set = [BitArray(contact)]
    objectives = Float64[]
    for i in 1:100
        new_active_set = Tuple{Float64, BitArray{2}}[]
        for contact in active_set
            result = c.MPC.solve!(model, state, contact_sequence=contact)
            if result.status != :Optimal
                result = c.MPC.solve!(model, state, contact_sequence=contact, relax=true)
            end
            push!(objectives, getvalue(getobjective(result.model.m)))
            features = vcat(vec(contact), result.model.m.linconstrDuals)
            _, decision_values = svmpredict(svm, reshape(features, length(features), 1))
            actions = svm.labels[reverse(sortperm(vec(decision_values)))]
#             actions = svm.labels[randperm(length(svm.labels))]
            for action in actions[1:5]
                contact = copy(contact)
                contact[action] = !contact[action]
                result = c.MPC.solve!(model, state, contact_sequence=contact)
                if result.status != :Optimal
                    result = c.MPC.solve!(model, state, contact_sequence=contact, relax=true)
                end
                push!(new_active_set, (getvalue(getobjective(result.model.m)), BitArray(contact)))
            end
        end
        active_set = [s[2] for s in sort(new_active_set)[1:5]]
    end
    objectives
end
                

svmbeam (generic function with 2 methods)

In [84]:
plot(svmbeam(svm, model, state, contact))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 90 
 
 
 95 
 
 
 100 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,3.93701 31.746,332.409 32.8884,223.349 34.0307,216.228 35.173,203.626 36.3154,188.321 37.4577,183.549 38.6001,183.549 39.7424,151.051 40.8848,151.051 
 42.0271,140.194 43.1694,384.952 44.3118,384.952 45.4541,384.952 46.5965,384.952 47.7388,335.182 48.8811,183.549 50.0235,183.549 51.1658,183.549 52.3082,183.549 
 53.4505,151.051 54.5929,384.952 55.7352,384.952 56.8775,384.952 58.0199,384.952 59.1622,384.952 60.3046,183.549 61.4469,183.549 62.5893,183.549 63.7316,183.549 
 64.8739,183.549 66.0163,384.952 67.1586,384.952 68.301,384.952 69.4433,384.952 70.5856,384.952 71.728,183.549 72.8703,183.549 74.0127,183.549 75.155,183.549 
 76.2974,183.549 77.4397,384.952 78.582,384.952 79.7244,384.952 80.8667,384.952 82.0091,384.952 83.1514,183.549 84.2938,183.549 85.4361,183.549 86.5784,183.549 
 87.7208,183.549 88.8631,384.952 90.0055,384.952 91.1478,384.952 92.2901,384.952 93.4325,384.952 94.5748,183.549 95.7172,183.549 96.8595,183.549 98.0019,183.549 
 99.1442,183.549 100.287,384.952 101.429,384.952 102.571,384.952 103.714,384.952 104.856,384.952 105.998,183.549 107.141,183.549 108.283,183.549 109.425,183.549 
 110.568,183.549 111.71,384.952 112.852,384.952 113.995,384.952 115.137,384.952 116.279,384.952 117.422,183.549 118.564,183.549 119.706,183.549 120.849,183.549 
 121.991,183.549 123.133,384.952 124.276,384.952 125.418,384.952 126.56,384.952 127.703,384.952 128.845,183.549 129.987,183.549 131.13,183.549 132.272,183.549 
 133.414,183.549 134.557,384.952 135.699,384.952 136.841,384.952 137.984,384.952 139.126,384.952 140.269,183.549 141.411,183.549 142.553,183.549 143.696,183.549 
 144.838,183.549 145.98,384.952 147.123,384.952 148.265,384.952 149.407,384.952 150.55,384.952 151.692,183.549 152.834,183.549 153.977,183.549 155.119,183.549 
 156.261,183.549 157.404,384.952 158.546,384.952 159.688,384.952 160.831,384.952 161.973,384.952 163.115,183.549 164.258,183.549 165.4,183.549 166.542,183.549 
 167.685,183.549 168.827,384.952 169.969,384.952 171.112,384.952 172.254,384.952 173.396,384.952 174.539,183.549 175.681,183.549 176.823,183.549 177.966,183.549 
 179.108,183.549 180.25,384.952 181.393,384.952 182.535,384.952 183.678,384.952 184.82,384.952 185.962,183.549 187.105,183.549 188.247,183.549 189.389,183.549 
 190.532,183.549 191.674,384.952 192.816,384.952 193.959,384.952 195.101,384.952 196.243,384.952 197.386,183.549 198.528,183.549 199.67,183.549 200.813,183.549 
 201.955,183.549 203.097,384.952 204.24,384.952 205.382,384.952 206.524,384.952 207.667,384.952 208.809,183.549 209.951,183.549 211.094,183.549 212.236,183.549 
 213.378,183.549 214.521,384.952 215.663,384.952 216.805,384.952 217.948,384.952 219.09,384.952 220.232,183.549 221.375,183.549 222.517,183.549 223.659,183.549 
 224.802,183.549 225.944,384.952 227.087,384.952 228.229,384.952 229.371,384.952 230.514,384.952 231.656,183.549 232.798,183.549 233.941,183.549 235.083,183.549 
 236.225,183.549 237.368,384.952 238.51,384.952 239.652,384.952 240.795,384.952 241.937,384.952 243.079,183.549 244.222,183.549 245.364,183.549 246.506,183.549 
 247.649,183.549 248.791,384.952 249.933,384.952 251.076,384.952 252.218,384.952 253.36,384.952 254.503,183.549 255.645,183.549 256.787,183.549 257.93,183.549 
 259.072,183.549 260.214,384.952 261.357,384.952 262.499,384.952 263.641,384.952 264.784,384.952 265.926,183.549 267.068,183.549 268.211,183.549 269.353,183.549 
 270.496,183.549 271.638,384.952 272.78,384.952 273.923,384.952 275.065,384.952 276.207,384.952 277.35,183.549 278.492,183.549 279.634,183.549 280.777,183.549 
 281.919,183.549 283.061,384.952 284.204,384.952 285.346,384.952 286.488,384.952 287.631,384.952 288.773,183.549 289.915,183.549 291.058,183.549 292.2,183.549 
 293.342,1

In [85]:
plot(svmbeam(svm, model, state, contact))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 90 
 
 
 95 
 
 
 100 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,3.93701 31.746,332.409 32.8884,223.349 34.0307,216.228 35.173,203.626 36.3154,188.321 37.4577,183.549 38.6001,183.549 39.7424,151.051 40.8848,151.051 
 42.0271,140.194 43.1694,384.952 44.3118,384.952 45.4541,384.952 46.5965,384.952 47.7388,335.182 48.8811,183.549 50.0235,183.549 51.1658,183.549 52.3082,183.549 
 53.4505,151.051 54.5929,384.952 55.7352,384.952 56.8775,384.952 58.0199,384.952 59.1622,384.952 60.3046,183.549 61.4469,183.549 62.5893,183.549 63.7316,183.549 
 64.8739,183.549 66.0163,384.952 67.1586,384.952 68.301,384.952 69.4433,384.952 70.5856,384.952 71.728,183.549 72.8703,183.549 74.0127,183.549 75.155,183.549 
 76.2974,183.549 77.4397,384.952 78.582,384.952 79.7244,384.952 80.8667,384.952 82.0091,384.952 83.1514,183.549 84.2938,183.549 85.4361,183.549 86.5784,183.549 
 87.7208,183.549 88.8631,384.952 90.0055,384.952 91.1478,384.952 92.2901,384.952 93.4325,384.952 94.5748,183.549 95.7172,183.549 96.8595,183.549 98.0019,183.549 
 99.1442,183.549 100.287,384.952 101.429,384.952 102.571,384.952 103.714,384.952 104.856,384.952 105.998,183.549 107.141,183.549 108.283,183.549 109.425,183.549 
 110.568,183.549 111.71,384.952 112.852,384.952 113.995,384.952 115.137,384.952 116.279,384.952 117.422,183.549 118.564,183.549 119.706,183.549 120.849,183.549 
 121.991,183.549 123.133,384.952 124.276,384.952 125.418,384.952 126.56,384.952 127.703,384.952 128.845,183.549 129.987,183.549 131.13,183.549 132.272,183.549 
 133.414,183.549 134.557,384.952 135.699,384.952 136.841,384.952 137.984,384.952 139.126,384.952 140.269,183.549 141.411,183.549 142.553,183.549 143.696,183.549 
 144.838,183.549 145.98,384.952 147.123,384.952 148.265,384.952 149.407,384.952 150.55,384.952 151.692,183.549 152.834,183.549 153.977,183.549 155.119,183.549 
 156.261,183.549 157.404,384.952 158.546,384.952 159.688,384.952 160.831,384.952 161.973,384.952 163.115,183.549 164.258,183.549 165.4,183.549 166.542,183.549 
 167.685,183.549 168.827,384.952 169.969,384.952 171.112,384.952 172.254,384.952 173.396,384.952 174.539,183.549 175.681,183.549 176.823,183.549 177.966,183.549 
 179.108,183.549 180.25,384.952 181.393,384.952 182.535,384.952 183.678,384.952 184.82,384.952 185.962,183.549 187.105,183.549 188.247,183.549 189.389,183.549 
 190.532,183.549 191.674,384.952 192.816,384.952 193.959,384.952 195.101,384.952 196.243,384.952 197.386,183.549 198.528,183.549 199.67,183.549 200.813,183.549 
 201.955,183.549 203.097,384.952 204.24,384.952 205.382,384.952 206.524,384.952 207.667,384.952 208.809,183.549 209.951,183.549 211.094,183.549 212.236,183.549 
 213.378,183.549 214.521,384.952 215.663,384.952 216.805,384.952 217.948,384.952 219.09,384.952 220.232,183.549 221.375,183.549 222.517,183.549 223.659,183.549 
 224.802,183.549 225.944,384.952 227.087,384.952 228.229,384.952 229.371,384.952 230.514,384.952 231.656,183.549 232.798,183.549 233.941,183.549 235.083,183.549 
 236.225,183.549 237.368,384.952 238.51,384.952 239.652,384.952 240.795,384.952 241.937,384.952 243.079,183.549 244.222,183.549 245.364,183.549 246.506,183.549 
 247.649,183.549 248.791,384.952 249.933,384.952 251.076,384.952 252.218,384.952 253.36,384.952 254.503,183.549 255.645,183.549 256.787,183.549 257.93,183.549 
 259.072,183.549 260.214,384.952 261.357,384.952 262.499,384.952 263.641,384.952 264.784,384.952 265.926,183.549 267.068,183.549 268.211,183.549 269.353,183.549 
 270.496,183.549 271.638,384.952 272.78,384.952 273.923,384.952 275.065,384.952 276.207,384.952 277.35,183.549 278.492,183.549 279.634,183.549 280.777,183.549 
 281.919,183.549 283.061,384.952 284.204,384.952 285.346,384.952 286.488,384.952 287.631,384.952 288.773,183.549 289.915,183.549 291.058,183.549 292.2,183.549 
 293.342,1

In [86]:
function svmsolve(svm::LIBSVM.SVM, model::c.MPC.MPCModel, state::c.MPC.State, contact)
    visited = Set([BitArray(contact)])
    objectives = Float64[]
    for i in 1:100
        result = c.MPC.solve!(model, state, contact_sequence=contact)
        if result.status != :Optimal
            result = c.MPC.solve!(model, state, contact_sequence=contact, relax=true)
        end
        push!(objectives, getvalue(getobjective(result.model.m)))

        features = vcat(vec(contact), result.model.m.linconstrDuals)
        _, decision_values = svmpredict(svm, reshape(features, length(features), 1))
        actions = svm.labels[reverse(sortperm(vec(decision_values)))]
        for action in actions
            contact[action] = !contact[action]
            b = BitArray(contact)
            if !(b in visited)
                push!(visited, b)
                break
            else
                contact[action] = !contact[action]  # put it back
            end
        end
    end
    objectives
end

svmsolve (generic function with 3 methods)

In [87]:
plot(svmsolve(svm, model, state, contact))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 90 
 
 
 95 
 
 
 100 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,15.9771 36.3154,199.963 42.0271,170.591 47.7388,3.93701 53.4505,11.704 59.1622,183.425 64.8739,219.907 70.5856,28.906 76.2974,39.7838 82.0091,231.768 
 87.7208,193.992 93.4325,21.3251 99.1442,12.4145 104.856,179.339 110.568,209.287 116.279,24.8082 121.991,130.235 127.703,293.74 133.414,257.527 139.126,111.115 
 144.838,139.339 150.55,291.415 156.261,335.474 161.973,166.59 167.685,153.486 173.396,321.861 179.108,278.722 184.82,127.265 190.532,100.922 196.243,246.955 
 201.955,282.648 207.667,119.517 213.378,145.233 219.09,309.698 224.802,272.279 230.514,124.858 236.225,157.775 241.937,311.321 247.649,357.585 253.36,187.564 
 259.072,212.834 264.784,383.75 270.496,335.512 276.207,180.707 281.919,143.268 287.631,291.586 293.342,330.099 299.054,164.812 304.766,51.128 310.477,238.44 
 316.189,205.823 321.901,36.5372 327.613,52.9866 333.324,230.424 339.036,274.575 344.748,78.2821 350.459,56.1228 356.171,250.484 361.883,209.473 367.595,34.3125 
 373.306,21.9801 379.018,190.029 384.73,221.334 390.441,35.3703 396.153,37.1054 401.865,223.054 407.577,191.764 413.288,23.724 419,36.019 424.712,211.156 
 430.423,252.137 436.135,57.7955 441.847,79.5391 447.559,275.803 453.27,231.705 458.982,54.3087 464.694,37.9507 470.405,207.209 476.117,239.799 481.829,52.514 
 487.54,166.126 493.252,331.398 498.964,292.906 504.676,144.604 510.387,181.961 516.099,336.745 521.811,384.952 527.522,214.05 533.234,189.203 538.946,359.216 
 544.658,312.969 550.369,159.435 556.081,126.562 561.793,273.974 567.504,311.381 573.216,146.925 578.928,121.295 584.64,284.428 590.351,248.741 596.063,102.706 
 
 "/>
 
 
 
 
 y1

In [88]:
function topn(n)
    [svm.labels[reverse(sortperm(decision_values[:, i]))[1:n]] for i in 1:size(decision_values, 2)]
end

topn (generic function with 1 method)

In [89]:
mean(getindex.(topn(1), 1) .== labels_test)

0.6514

In [90]:
t = topn(5)
mean([labels_test[i] in t[i] for i in 1:length(labels_test)])

0.9802

In [91]:
histogram(labels)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [92]:
histogram(predicted_labels)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1